![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/5.2.Spark_OCR_Deidentification.ipynb)

# Spark OCR 


## Blogposts and videos

- [How to Setup Spark OCR on UBUNTU - Video](https://www.youtube.com/watch?v=cmt4WIcL0nI)

- [Installing Spark NLP and Spark OCR in air-gapped networks (offline mode)
](https://medium.com/spark-nlp/installing-spark-nlp-and-spark-ocr-in-air-gapped-networks-offline-mode-f42a1ee6b7a8)

- [Table Detection & Extraction in Spark OCR](https://medium.com/spark-nlp/table-detection-extraction-in-spark-ocr-50765c6cedc9)

- [Signature Detection in Spark OCR](https://medium.com/spark-nlp/signature-detection-in-spark-ocr-32f9e6f91e3c)

- [GPU image pre-processing in Spark OCR](https://medium.com/spark-nlp/gpu-image-pre-processing-in-spark-ocr-3-1-0-6fc27560a9bb)

**More examples here**

https://github.com/JohnSnowLabs/spark-ocr-workshop

**Colab Setup**

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_ocr.json')

with open('spark_ocr.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.2.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
!pip -q install --upgrade spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark OCR
! pip install spark-ocr==$OCR_VERSION --extra-index-url=https://pypi.johnsnowlabs.com/$SPARK_OCR_SECRET --upgrade

<b><h1><font color='darkred'>!!! ATTENTION !!! </font><h1><b>

<b>After running previous cell, <font color='darkred'>RESTART the COLAB RUNTIME </font> and go ahead.<b>

In [ ]:
import json, os

with open("spark_ocr.json", 'r') as f:
  license_keys = json.load(f)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

# Defining license key-value pairs as local variables
locals().update(license_keys)

In [ ]:
import sparkocr
import sys
from pyspark.sql import SparkSession
from sparkocr import start
import os
import base64
from sparkocr.transformers import *
from pyspark.ml import PipelineModel
from pyspark.sql import functions as F
from sparkocr.enums import *
from sparkocr.utils import display_images

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *

In [ ]:
spark = sparkocr.start(secret=SPARK_OCR_SECRET, 
                       nlp_version=PUBLIC_VERSION,
                       nlp_secret=SECRET,
                       nlp_internal=JSL_VERSION
                       )
spark

## Dicom Image Deidentifier 

**Define deidentification pipeline**

In [ ]:
def deidentification_nlp_pipeline(input_column, prefix = ""):
    document_assembler = DocumentAssembler() \
        .setInputCol(input_column) \
        .setOutputCol(prefix + "document")

    # Sentence Detector annotator, processes various sentences per line
    sentence_detector = SentenceDetector() \
        .setInputCols([prefix + "document"]) \
        .setOutputCol(prefix + "sentence")

    tokenizer = Tokenizer() \
        .setInputCols([prefix + "sentence"]) \
        .setOutputCol(prefix + "token")

    # Clinical word embeddings
    word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models") \
        .setInputCols([prefix + "sentence", prefix + "token"]) \
        .setOutputCol(prefix + "embeddings")
    # NER model trained on i2b2 (sampled from MIMIC) dataset
    clinical_ner = MedicalNerModel.pretrained("ner_deid_large", "en", "clinical/models") \
        .setInputCols([prefix + "sentence", prefix + "token", prefix + "embeddings"]) \
        .setOutputCol(prefix + "ner")

    custom_ner_converter = NerConverter() \
        .setInputCols([prefix + "sentence", prefix + "token", prefix + "ner"]) \
        .setOutputCol(prefix + "ner_chunk") \
        .setWhiteList(['NAME', 'AGE', 'CONTACT', 'LOCATION', 'PROFESSION', 'PERSON', 'DATE'])

    nlp_pipeline = Pipeline(stages=[
            document_assembler,
            sentence_detector,
            tokenizer,
            word_embeddings,
            clinical_ner,
            custom_ner_converter
        ])
    empty_data = spark.createDataFrame([[""]]).toDF(input_column)
    nlp_model = nlp_pipeline.fit(empty_data)
    return nlp_model

**Define OCR transformers and pipeline**

In [ ]:
# Read dicom as image
dicom_to_image = DicomToImage() \
    .setInputCol("content") \
    .setOutputCol("image_raw") \
    .setMetadataCol("metadata") \
    .setDeIdentifyMetadata(True)

adaptive_thresholding = ImageAdaptiveThresholding() \
    .setInputCol("image_raw") \
    .setOutputCol("corrected_image") \
    .setBlockSize(47) \
    .setOffset(4) \
    .setKeepInput(True)

# Extract text from image
ocr = ImageToText() \
    .setInputCol("corrected_image") \
    .setOutputCol("text")

# Found coordinates of sensitive data
position_finder = PositionFinder() \
    .setInputCols("ner_chunk") \
    .setOutputCol("coordinates") \
    .setPageMatrixCol("positions") \
    .setMatchingWindow(100) \
    .setPadding(1)

# Found sensitive data using DeIdentificationModel
deidentification_rules = DeIdentificationModel.pretrained("deidentify_rb_no_regex", "en", "clinical/models") \
    .setInputCols(["metadata_sentence", "metadata_token", "metadata_ner_chunk"]) \
    .setOutputCol("deidentified_metadata_raw")

finisher = Finisher() \
    .setInputCols(["deidentified_metadata_raw"]) \
    .setOutputCols("deidentified_metadata") \
    .setOutputAsArray(False) \
    .setValueSplitSymbol("") \
    .setAnnotationSplitSymbol("")

# Draw filled rectangle for hide sensitive data
drawRegions = ImageDrawRegions()  \
    .setInputCol("image_raw")  \
    .setInputRegionsCol("coordinates")  \
    .setOutputCol("image_with_regions")  \
    .setFilledRect(True) \
    .setRectColor(Color.black)

# Store image back to Dicom document
imageToDicom = ImageToDicom() \
    .setInputCol("image_with_regions") \
    .setOutputCol("dicom") 
    
# OCR pipeline
deid_pipeline = PipelineModel(stages=[
    dicom_to_image,
    adaptive_thresholding,
    ocr,
    deidentification_nlp_pipeline(input_column="text"),
    position_finder,
    drawRegions,
    #imageToDicom  # Commented for able to demonstrate intermidiate results before aggregation
])

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-ocr-workshop/master/jupyter/data/dicom/deidentify-medical-2.dcm  -P content/dicom/
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-ocr-workshop/master/jupyter/data/dicom/deidentify-brains-front-medical-3.dcm -P content/dicom/


file_path='content/dicom/*.dcm'
dicom_df = spark.read.format("binaryFile").load(file_path)
dicom_df.show()

In [ ]:
display_images(DicomToImage().transform(dicom_df))

In [ ]:
deid_results = deid_pipeline.transform(dicom_df).cache()

In [ ]:
from sparkocr.utils import display_image, to_pil_image
import matplotlib.pyplot as plt

for r in deid_results.select("image_raw", "image_with_regions").collect():
  img_orig = r.image_raw
  img_deid = r.image_with_regions

  img_pil_orig = to_pil_image(img_orig, img_orig.mode)
  img_pil_deid = to_pil_image(img_deid, img_deid.mode)

  plt.figure(figsize=(24,16))
  plt.subplot(1, 2, 1)
  plt.imshow(img_pil_orig, cmap='gray')
  plt.title('original')
  plt.subplot(1, 2, 2)
  plt.imshow(img_pil_deid, cmap='gray')
  plt.title("de-id'd")
  plt.show()

## PDF Document Deidentifier 

In [ ]:
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession

from sparkocr.transformers import *
from sparkocr.enums import *
from sparkocr.utils import display_image, display_images
from sparkocr.metrics import score
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl

In [ ]:
from sparkocr.transformers import *
from sparkocr.utils import display_image, to_pil_image

**Define OCR transformers and pipeline**

In [ ]:
# Read Pdf as image
pdf_to_image = PdfToImage()\
    .setInputCol("content")\
    .setOutputCol("image_raw")\
    .setResolution(400)

# Extract text from image
ocr = ImageToText() \
    .setInputCol("image_raw") \
    .setOutputCol("text") \
    .setIgnoreResolution(False) \
    .setPageIteratorLevel(PageIteratorLevel.SYMBOL) \
    .setPageSegMode(PageSegmentationMode.SPARSE_TEXT) \
    .setWithSpaces(True) \
    .setConfidenceThreshold(70)

hocr = ImageToHocr() \
    .setInputCol("image_raw") \
    .setOutputCol("hocr") \
    .setIgnoreResolution(False) \
    .setOcrParams(["preserve_interword_spaces=0"])\
    .setPageIteratorLevel(PageIteratorLevel.SYMBOL)\
    .setPageSegMode(PageSegmentationMode.SPARSE_TEXT) \

# Found coordinates of sensitive data
position_finder = PositionFinder() \
    .setInputCols("ner_chunk") \
    .setOutputCol("coordinates") \
    .setPageMatrixCol("positions") \
    .setMatchingWindow(100) \
    .setPadding(1)

# Draw filled rectangle for hide sensitive data
drawRegions = ImageDrawRegions()  \
    .setInputCol("image_raw")  \
    .setInputRegionsCol("coordinates")  \
    .setOutputCol("image_with_regions")  \
    .setFilledRect(True) \
    .setRectColor(Color.black)
    
# OCR pipeline
deid_pipeline = PipelineModel(stages=[
    pdf_to_image,
    ocr,
    hocr,
    deidentification_nlp_pipeline(input_column="text"),
    position_finder,
    drawRegions
])

In [ ]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ocr/MT_00.pdf -P content/deid/


In [ ]:
import pkg_resources
image_path = pkg_resources.resource_filename('sparkocr', 'resources/ocr/images/p1.jpg')
image_df = spark.read.format("binaryFile").load(image_path)

In [ ]:
file_path='content/deid/*.pdf'
pdfs = spark.read.format("binaryFile").load(file_path)
pdfs.show()

In [ ]:
deid_results = deid_pipeline.transform(pdfs).cache()

In [ ]:
deid_results.select("ner_chunk").show(truncate=False)

In [ ]:
deid_results.select('coordinates').show(truncate=False)

In [ ]:
!wget -q -O jsl_utils.py https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/utils/jsl_utils.py

In [ ]:
from sparkocr.utils import display_image, to_pil_image
import matplotlib.pyplot as plt

r = deid_results.select("image_raw", "image_with_regions").collect()[0]
img_orig = r.image_raw
img_deid = r.image_with_regions

img_pil_orig = to_pil_image(img_orig, img_orig.mode)
img_pil_deid = to_pil_image(img_deid, img_deid.mode)

plt.figure(figsize=(24,16))
plt.subplot(1, 2, 1)
plt.imshow(img_pil_orig, cmap='gray')
plt.title('original')
plt.subplot(1, 2, 2)
plt.imshow(img_pil_deid, cmap='gray')
plt.title("de-id'd")
plt.show()

In [ ]:
from jsl_utils import *

from PIL import Image

image_list = []  # append images

for row in range(0,1):
    text = deid_results.select('text').collect()[row][0]
    hocr =  deid_results.select('hocr').collect()[row][0]

    lmodel = LightPipeline(deidentification_nlp_pipeline('text'))
    ner_result = lmodel.fullAnnotate(text)
    ent_dict_list = [{'begin':x.begin, 'end':x.end, 'chunk':x.result, 'ner_label':x.metadata['entity'], 'sentence_id':x.metadata['sentence']} for x in ner_result[0]['ner_chunk']]
    
    coord_df = get_coordinates_frame(ent_dict_list, text, hocr)

    img_deid = deid_results.select('image_raw').collect()[row][0]
    img_pil_orig = to_pil_image(img_deid, img_deid.mode)
    img_pil_orig = img_pil_orig.convert("RGBA")

    img_deid = deid_results.select('image_raw').collect()[row][0]
    img_pil_deid = to_pil_image(img_deid, img_deid.mode)
    img_pil_deid = img_pil_deid.convert("RGBA")

    draw_outline(img_pil_deid, coord_df)

In [ ]:
from PIL import Image

highlighted_image_list = []  # append highlighted images

for row in range(0,1):
    text = deid_results.select('text').collect()[row][0]
    hocr =  deid_results.select('hocr').collect()[row][0]

    lmodel = LightPipeline(deidentification_nlp_pipeline('text'))
    ner_result = lmodel.fullAnnotate(text)
    ent_dict_list = [{'begin':x.begin, 'end':x.end, 'chunk':x.result, 'ner_label':x.metadata['entity'], 'sentence_id':x.metadata['sentence']} for x in ner_result[0]['ner_chunk']]
    
    coord_df = get_coordinates_frame(ent_dict_list, text, hocr)

    img_deid = deid_results.select('image_raw').collect()[row][0]
    img_pil_orig = to_pil_image(img_deid, img_deid.mode)
    img_pil_orig = img_pil_orig.convert("RGBA")

    img_deid = deid_results.select('image_raw').collect()[row][0]
    img_pil_deid = to_pil_image(img_deid, img_deid.mode)
    img_pil_deid = img_pil_deid.convert("RGBA")

    highlight(img_pil_deid, coord_df)